In [14]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, Masking, Bidirectional
from keras.regularizers import l2, l1
import numpy as np
from readdata import read_data
import collections
from keras import optimizers
from keras.callbacks import ModelCheckpoint

save_dir = 'saves'

x_train, y_train, x_text, y_test = read_data(level=0, length_limit=350)

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for i, (input_text, target) in enumerate(zip(x_train, y_train)):
    input_texts.append(input_text)
    target_text = '\t' + input_text + '\n'
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
print('Class counts:', [(key, value) for
                        key, value in (collections.Counter(y_train)).iteritems()])

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, -(t+1), input_token_index[char]] = 1.   #-(t+1)
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

('Missing proteins: ', 174856)
('All count: ', 184939)
('Number of samples:', 3690)
('Number of unique input tokens:', 21)
('Number of unique output tokens:', 23)
('Max sequence length for inputs:', 350)
('Max sequence length for outputs:', 352)
('Class counts:', [('1', 857), ('3', 1028), ('2', 717), ('5', 85), ('4', 313), ('6', 690)])


In [15]:
print encoder_input_data.shape
print decoder_input_data.shape
print decoder_target_data.shape

(3690, 350, 21)
(3690, 352, 23)
(3690, 352, 23)


In [16]:
batch_size = 128  # Batch size for training.
epochs = 200  # Number of epochs to train for.
latent_dim = 32  # Latent dimensionality of the encoding space.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Callbacks
checkpointer = ModelCheckpoint('checkpoints/encoder-decoder-model.hdf5', 
                               verbose=0, save_weights_only=False)

# Run training
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
adadlt = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
model.compile(optimizer='adam', loss='categorical_crossentropy')
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size, callbacks = [checkpointer],
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 2952 samples, validate on 738 samples
Epoch 1/200
2952/2952 [==============================] - 43s - loss: 2.2941 - val_loss: 1.7873
Epoch 2/200
2952/2952 [==============================] - 42s - loss: 2.1936 - val_loss: 1.7255
Epoch 3/200
2952/2952 [==============================] - 41s - loss: 2.1583 - val_loss: 1.7120
Epoch 4/200
2952/2952 [==============================] - 41s - loss: 2.1481 - val_loss: 1.7082
Epoch 5/200
2952/2952 [==============================] - 42s - loss: 2.1441 - val_loss: 1.7061
Epoch 6/200
2952/2952 [==============================] - 42s - loss: 2.1412 - val_loss: 1.7049
Epoch 7/200
2952/2952 [==============================] - 41s - loss: 2.1378 - val_loss: 1.7045
Epoch 8/200
2952/2952 [==============================] - 39s - loss: 2.1351 - val_loss: 1.7017
Epoch 9/200
2952/2952 [==============================] - 41s - loss: 2.1324 - val_loss: 1.6998
Epoch 10/200
2952/2952 [==============================] - 41s - loss: 2.1297 - val_loss: 1.6997
Ep

2952/2952 [==============================] - 39s - loss: 2.0464 - val_loss: 1.6751
Epoch 86/200
2952/2952 [==============================] - 39s - loss: 2.0437 - val_loss: 1.6740
Epoch 87/200
2952/2952 [==============================] - 39s - loss: 2.0427 - val_loss: 1.6736
Epoch 88/200
2952/2952 [==============================] - 39s - loss: 2.0418 - val_loss: 1.6741
Epoch 89/200
2952/2952 [==============================] - 39s - loss: 2.0411 - val_loss: 1.6734
Epoch 90/200
2952/2952 [==============================] - 39s - loss: 2.0400 - val_loss: 1.6758
Epoch 91/200
2952/2952 [==============================] - 39s - loss: 2.0412 - val_loss: 1.6732
Epoch 92/200
2952/2952 [==============================] - 39s - loss: 2.0390 - val_loss: 1.6740
Epoch 93/200
2952/2952 [==============================] - 39s - loss: 2.0377 - val_loss: 1.6738
Epoch 94/200
2952/2952 [==============================] - 39s - loss: 2.0377 - val_loss: 1.6751
Epoch 95/200
2952/2952 [=============================

2952/2952 [==============================] - 38s - loss: 1.9839 - val_loss: 1.6617
Epoch 170/200
2952/2952 [==============================] - 38s - loss: 1.9893 - val_loss: 1.6639
Epoch 171/200
2952/2952 [==============================] - 37s - loss: 1.9944 - val_loss: 1.6656
Epoch 172/200
2952/2952 [==============================] - 38s - loss: 1.9866 - val_loss: 1.6610
Epoch 173/200
2952/2952 [==============================] - 37s - loss: 1.9864 - val_loss: 1.6604
Epoch 174/200
2952/2952 [==============================] - 37s - loss: 1.9821 - val_loss: 1.6608
Epoch 175/200
2952/2952 [==============================] - 37s - loss: 1.9807 - val_loss: 1.6608
Epoch 176/200
2952/2952 [==============================] - 37s - loss: 1.9798 - val_loss: 1.6596
Epoch 177/200
2952/2952 [==============================] - 36s - loss: 1.9792 - val_loss: 1.6605
Epoch 178/200
2952/2952 [==============================] - 36s - loss: 1.9783 - val_loss: 1.6607
Epoch 179/200
2952/2952 [===================

In [ ]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

# save encoder model
encoder_model_yaml = encoder_model.to_yaml()
with open("encoder_model.yaml", "w") as yaml_file:
    yaml_file.write(encoder_model_yaml)
encoder_model.save('encoder_model.h5')
print("Saved model to disk")


decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence